In [ ]:

from google.colab import drive

drive.mount('/content/drive')  
  

# New section

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:

import os

arr = os.listdir("/content/drive/MyDrive/text_summarization/tech")

arr.sort()

arr  


FileNotFoundError: ignored

In [ ]:

import spacy
!python -m spacy download en_core_web_lg 


In [ ]:

import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import sent_tokenize

import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize   

import spacy
nlp = spacy.load("en_core_web_lg") 

import networkx as nx


In [ ]:

def process_text(text):
    doc = nlp(text.lower())
    result = []
    for token in doc:
        if token.text in nlp.Defaults.stop_words:
            continue
        if token.is_punct:
            continue
        if token.lemma_ == '-PRON-' :
            continue
        result.append(token.lemma_)
    return " ".join(result)
    

In [ ]:

def spacy_similarity(s1,s2):
  doc1 = nlp(s1)
  doc2 = nlp(s2)
  return float(doc1.similarity(doc2))   
 


In [ ]:

for t in range(0,len(arr)):
  f = open(r'/content/drive/MyDrive/text_summarization/tech/'+arr[t],'r')   
  lst = []
  pre_lst = f.read().split('\n')
  for i in range(len(pre_lst)):
    if(len(pre_lst[i]) != 0):
      lst.append(pre_lst[i])

  sentences = ''
  for i in range(len(lst)):
    if(lst[i][len(lst[i])-1] != '.'):
      sentences = sentences + lst[i] + '. '
    else :
      sentences = sentences + lst[i] + ' '

  text = sentences   

  sentences1 = sent_tokenize(text)  

  sentences = []

  for i in range(0,len(sentences1)):
    word_tokens = word_tokenize(sentences1[i])
  
  
    filtered_sentence = ""
  
    for w in word_tokens:
        if w not in stop_words:
            if(filtered_sentence == ""):
              filtered_sentence = w
            else :
              filtered_sentence = filtered_sentence + " " + w

    sentences.append(filtered_sentence)

  for i in range(len(sentences)):
    sentences[i] = process_text(sentences[i])  

  edges = [(i,j,spacy_similarity(x,y)) 
         for i,x in enumerate(sentences) 
         for j,y in enumerate(sentences) if i != j]   

  G = nx.DiGraph()
  G.add_weighted_edges_from(edges)

  nst = {n: 1 for n in G.nodes}
  pr = nx.pagerank(G, nstart=nst)

  avg = 0
  for i in range(len(pr)):
    avg = avg + pr[i]

  avg = avg / len(pr)

  avg2 = 0
  count = 0  

  lst = []

  for i in range(len(pr)):
    lst.append(pr[i])
    if(pr[i] >= avg):
      avg2 = avg2 + pr[i]
      count+=1

  avg2 = avg2 / count 

  sd = statistics.stdev(lst)

  sum = ""

  count = 0

  for i in range(len(sentences)):
    if(pr[i] >= (avg2 + 0.1 * sd)):
      sum = sum + sentences1[i]
      count = count + 1   


  with open(r'/content/drive/MyDrive/text_summarization/tech_summary_model/'+arr[t], 'w') as f:
    f.write(str(sum))



Streaming output truncated to the last 5000 lines.
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluat

# rouge score 

In [ ]:

!pip install rouge 


In [ ]:

from rouge import Rouge

avg_score = 0

for i in range(len(arr)):
  f1 = open(r'/content/drive/MyDrive/text_summarization/tech_summary/'+arr[i],'r')  
  original = f1.read()
  
  f2 = open(r'/content/drive/MyDrive/text_summarization/tech_summary_model/'+arr[i],'r')  
  model_generated = f2.read()

  r = Rouge()
  a = r.get_scores(original, model_generated)[0]['rouge-l']['f']
  avg_score = avg_score + a

  with open(r'/content/drive/MyDrive/text_summarization/tech_summary_score/'+arr[i], 'w') as f3:
    f3.write(str(a))

avg_score = avg_score/(len(arr))

print(avg_score)   


0.6386446617079882
